In [1]:
import random
import numpy as np
from collections import defaultdict
import os
import sys
from time import time

In [2]:
class Member:
    def __init__(self, r_d, label=None, doc_id=None):
        self._r_d = r_d
        self._label = label
        self._doc_id = doc_id

In [3]:
class Cluster:
    def __init__(self):
        self._centroid = None
        self._members = []

    def reset_members(self):
        self._members = []

    def add_member(self, member):
        self._members.append(member)

In [4]:
class Kmeans:
    def __init__(self, num_clusters):
        self._num_clusters = num_clusters
        self._clusters = [Cluster() for _ in range(self._num_clusters)]

    def load_data(self, data_path):
        def sparse_to_dense(sparse_r_d, vocab_size):
            r_d = [0.0 for _ in range(vocab_size)]
            indices_tfidfs = sparse_r_d.split()
            for index_tfidf in indices_tfidfs:
                index = int(index_tfidf.split(":")[0])
                tfidf = float(index_tfidf.split(":")[1])
                r_d[index] = tfidf
            return np.array(r_d)

        with open(data_path) as f:
            d_lines = f.read().splitlines()

        with open(os.getcwd()+"/20news-bydate/20news-full-words-idfs.txt") as f:
            vocab_size = len(f.read().splitlines())

        self._data = []
        self._label_count = defaultdict(int)
        for d in d_lines:
            features = d.split("<fff>")
            label, doc_id = int(features[0]), int(features[1])
            if label not in self._label_count:
                print(f"Loading cluster {label}")
            self._label_count[label] += 1
            r_d = sparse_to_dense(sparse_r_d=features[2], vocab_size=vocab_size)
            self._data.append(Member(r_d=r_d, label=label, doc_id=doc_id))

    def compute_similarity(self, a, b):
        return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

    def random_init(self, seed_value_id):
        start_time = time()
        set_candidates = set(range(len(self._data)))
        set_candidates.remove(seed_value_id)
        print(f"Seed centroid: {seed_value_id}")
        E = [self._data[seed_value_id]._r_d]
        while len(E) < self._num_clusters:
            new_centroid_id = None
            min_similarity_val = 1
            for i in set_candidates:
                local_max_similarity = -1
                for centroid in E:
                    local_max_similarity = max(local_max_similarity, self.compute_similarity(self._data[i]._r_d, centroid))
                if local_max_similarity < min_similarity_val:
                    new_centroid_id = i
                    min_similarity_val = local_max_similarity
            if new_centroid_id:
                print(f"Got new centroid {len(E)}: {new_centroid_id}")
                set_candidates.remove(new_centroid_id)
                E.append(self._data[new_centroid_id]._r_d)
            else:
                raise Exeption("Could not find enough centroids")
        for i in range(len(self._clusters)):
            self._clusters[i]._centroid = E[i]
        end_time = time()
        print("Execution time: ", end_time - start_time)

    def select_cluster_for(self, member):
        best_fit_cluster = None
        max_similarity = -1
        for cluster in self._clusters:
            similarity = self.compute_similarity(member._r_d, cluster._centroid)
            if similarity > max_similarity:
                best_fit_cluster = cluster
                max_similarity = similarity
        best_fit_cluster.add_member(member)
        return max_similarity

    def update_centroid_of(self, cluster):
        aver_r_d = np.mean([member._r_d for member in cluster._members], axis=0)
        new_centroid = aver_r_d/np.linalg.norm(aver_r_d)
        cluster._centroid = new_centroid
        return cluster._centroid

    def stopping_condition(self, criterion, threshold):
        criteria = ["max_iters", "centroid", "similarity"]
        assert criterion in criteria
        if criterion == "max_iters":
            print(f"Iterations: {self._iteration}")
            if self._iteration >= threshold:
                return True
            return False

        elif criterion == "centroid":        
            E_new_minus_E = [centroid for centroid in self._E_new if centroid not in self._E]
            print(f"Number centroid changed: {len(E_new_minus_E)}")
            if len(E_new_minus_E) <= threshold:
                return True
            return False
            
        else:
            S_new_minus_S = self._S_new - self._S
            print(f"S changed: {S_new_minus_S}\tOld S: {self._S}\tNew S: {self._S_new}")
            if abs(S_new_minus_S) <= threshold:
                return True
            return False

    def run(self, seed_value_id, criterion, threshold):
        self.random_init(seed_value_id)
        self._iteration = 0
        self._S = 0
        self._E = []

        while True:
            self._iteration += 1
            print("Processing iteration:", self._iteration)
            for cluster in self._clusters:
                cluster.reset_members()
            
            self._S_new = 0
            for member in self._data:
                self._S_new += self.select_cluster_for(member)

            self._E_new = []
            for cluster in self._clusters:
                self._E_new.append(list(self.update_centroid_of(cluster)))

            if self.stopping_condition(criterion, threshold):
                print("Catch stopping condition by criterion:", criterion)
                break

            self._S = self._S_new
            self._E = self._E_new

    def compute_purity(self, num_clusters):
        majority_sum = 0
        for cluster in self._clusters:
            member_labels = [member._label for member in cluster._members]
            max_count = max([member_labels.count(label) for label in range(num_clusters)])
            majority_sum += max_count
        return majority_sum * 1 / len(self._data)

    def compute_NMI(self, num_clusters):
        I_value, H_O, H_C, N = 0.0, 0.0, 0.0, len(self._data)

        for cluster in self._clusters:
            wk = len(cluster._members) * 1.0
            H_O += -wk / N * np.log10(wk / N)

        for label in range(num_clusters):
            cj = self._label_count[label] * 1.0
            H_C += -cj / N * np.log10(cj / N)

        for cluster in self._clusters:
            member_labels = [member._label for member in cluster._members]
            for label in range(num_clusters):
                wk_cj = member_labels.count(label) * 1.0
                wk = len(cluster._members) * 1.0
                cj = self._label_count[label] * 1.0
                I_value += wk_cj / N * np.log10(N * wk_cj / (wk * cj) + 1e-12)

        return I_value * 2.0 / (H_C + H_O)

In [5]:
# Create kmeans sample
num_clusters = 20
k = Kmeans(num_clusters)

In [6]:
# Load data
k.load_data(os.getcwd()+"/20news-bydate/20news-full-tf-idf.txt")

Loading cluster 0
Loading cluster 1
Loading cluster 2
Loading cluster 3
Loading cluster 4
Loading cluster 5
Loading cluster 6
Loading cluster 7
Loading cluster 8
Loading cluster 9
Loading cluster 10
Loading cluster 11
Loading cluster 12
Loading cluster 13
Loading cluster 14
Loading cluster 15
Loading cluster 16
Loading cluster 17
Loading cluster 18
Loading cluster 19


In [7]:
# Clusters 
print(f"Given number clusters: {num_clusters}")
print(f"Number clusters: {len(k._label_count)}")
print(f"Label dictionary: {k._label_count}")

Given number clusters: 20
Number clusters: 20
Label dictionary: defaultdict(<class 'int'>, {0: 799, 1: 973, 2: 985, 3: 982, 4: 963, 5: 988, 6: 975, 7: 990, 8: 996, 9: 994, 10: 999, 11: 991, 12: 984, 13: 990, 14: 987, 15: 997, 16: 910, 17: 940, 18: 775, 19: 628})


In [8]:
# Running
k.run(seed_value_id=random.choice(range(len(k._data))), criterion="similarity", threshold=1e-2)

Seed centroid: 4628
Got new centroid 1: 6200
Got new centroid 2: 139
Got new centroid 3: 12042
Got new centroid 4: 2046
Got new centroid 5: 1275
Got new centroid 6: 10395
Got new centroid 7: 6314
Got new centroid 8: 8286
Got new centroid 9: 14164
Got new centroid 10: 15459
Got new centroid 11: 16775
Got new centroid 12: 13326
Got new centroid 13: 13402
Got new centroid 14: 2245
Got new centroid 15: 2990
Got new centroid 16: 3960
Got new centroid 17: 8658
Got new centroid 18: 16965
Got new centroid 19: 2323
Execution time:  110.56937074661255
Processing iteration: 1
S changed: 1036.8676577750523	Old S: 0	New S: 1036.8676577750523
Processing iteration: 2
S changed: 2150.802472254799	Old S: 1036.8676577750523	New S: 3187.6701300298514
Processing iteration: 3
S changed: 285.8014166120711	Old S: 3187.6701300298514	New S: 3473.4715466419225
Processing iteration: 4
S changed: 97.45088282832785	Old S: 3473.4715466419225	New S: 3570.9224294702503
Processing iteration: 5
S changed: 45.6813819708

In [9]:
# Measurements
print(f"Purity: {k.compute_purity(num_clusters)}")
print(f"NMI: {k.compute_NMI(num_clusters)}")

Purity: 0.4178074923060596
NMI: 0.4648891524916016
